In [ ]:
from modulo_cargar_datos import cargar_datos
from modulo_procesamiento_de_data import procesar_gestion_campo, procesar_bases_enviadas, procesar_seriales_recuperados, procesar_canales_alternos
from modulo_cruzar_datos import cruzar_datos

# Cargar los datos
gestion_campo, bases_enviadas, seriales_recuperados, canales_alternos, churn, desconexiones = cargar_datos()

# Procesar los datos
gestion_campo = procesar_gestion_campo(gestion_campo)
bases_enviadas = procesar_bases_enviadas(bases_enviadas)
seriales_recuperados = procesar_seriales_recuperados(seriales_recuperados)
canales_alternos = procesar_canales_alternos(canales_alternos)

# Imprimir información sobre los DataFrames procesados
print("La fecha min de gestion de campo es: ", gestion_campo["Fecha de cierre"].min())
print("La fecha max de gestion de campo es: ", gestion_campo["Fecha de cierre"].max())
gestion_campo.info()

print("La fecha min de bases enviadas es: ", bases_enviadas["FECHA_ENVIO IQE"].min())
print("La fecha max de bases enviadas es: ", bases_enviadas["FECHA_ENVIO IQE"].max())
bases_enviadas.info()

print("La fecha min de seriales_recuperados es: ", seriales_recuperados["Fecha"].min())
print("La fecha max de seriales_recuperados es: ", seriales_recuperados["Fecha"].max())
seriales_recuperados.info()

print("La fecha min de canales_alternos es: ", canales_alternos["FECHA"].min())
print("La fecha max de canales_alternos es: ", canales_alternos["FECHA"].max())
canales_alternos.info()

In [ ]:
#Cruzar datos
# Llamar a la función cruzar_datos
gestion_campo_v3 = cruzar_datos(gestion_campo, bases_enviadas, seriales_recuperados)
print(gestion_campo_v3.info())

# CRUCE GESTION CAMPO VS BASES ENVIADAS
LA FUNCIONALIDAD ES TRAERNOS LOS CAMPOS QUE SE REQUIEREN DE BASES ENVIADAS QUE NO TRAE LA TABLA DE GESTION EN CAMPO COMO: MACROMOTIVO, UNIDAD DE NEGOCIO, FECHA ENVIO Y PRODUCTO)

#### LLAVE PC


In [7]:
#creamos una columa llamada cruce con el valor fijo de NO CRUZO
gestion_campo["CRUCE"]="NO CRUZO"
gestion_campo.loc[gestion_campo["PC"].isin(bases_enviadas["PC"]), "CRUCE" ] = "CRUZO PC"
gestion_pc= pd.merge(gestion_campo,
                     bases_enviadas[["PC","MACRO MOTIVO","UNIDAD_DE_NEGOCIO","FECHA_ENVIO IQE","PRODUCTO","TIPO BASE","NombreArchivo"]],
                     left_on="PC",right_on="PC",how="inner")
gestion_pc=gestion_pc.drop_duplicates(subset="PCS")
gestion_pc.reset_index(drop=True,inplace=True)
print(gestion_campo["CRUCE"].value_counts())
print(gestion_pc.info())

CRUCE
NO CRUZO    480394
CRUZO PC    407863
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407863 entries, 0 to 407862
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   No. Contrato       407863 non-null  object        
 1   Cédula usuario     407863 non-null  object        
 2   Dirección          407863 non-null  object        
 3   Ciudad             407863 non-null  object        
 4   Departamento       407863 non-null  object        
 5   Fecha de cierre    407863 non-null  datetime64[ns]
 6   Hora               407863 non-null  object        
 7   Tipo de cierre     407863 non-null  object        
 8   Serial             407863 non-null  object        
 9   Prioridad          407863 non-null  object        
 10  PCS                407863 non-null  object        
 11  PC                 407863 non-null  object        
 12  dicc_prioridades   406869 non-

#### LlAVE CC
* La informacion que no me cruzo por pc la cruzaremos vs bases enviadads por cc

In [8]:
gestion_cc=gestion_campo.loc[gestion_campo["CRUCE"]=="NO CRUZO"]
gestion_cc.loc[gestion_cc["Cédula usuario"].isin(bases_enviadas["CLIENTE_ID"]), "CRUCE" ] = "CRUZO CEDULA"
gestion_cc= pd.merge(gestion_cc,
                     bases_enviadas[["CLIENTE_ID","MACRO MOTIVO","UNIDAD_DE_NEGOCIO","FECHA_ENVIO IQE","PRODUCTO","TIPO BASE","NombreArchivo"]],
                     left_on="Cédula usuario",right_on="CLIENTE_ID",how="left")
gestion_cc=gestion_cc.drop_duplicates(subset="PCS")
gestion_cc.drop(columns="CLIENTE_ID",inplace=True)
gestion_cc.reset_index(drop=True,inplace=True)
print(gestion_cc["CRUCE"].value_counts())
print(gestion_cc.info())

CRUCE
NO CRUZO        433349
CRUZO CEDULA     47045
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480394 entries, 0 to 480393
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   No. Contrato       480394 non-null  object        
 1   Cédula usuario     480394 non-null  object        
 2   Dirección          480394 non-null  object        
 3   Ciudad             480394 non-null  object        
 4   Departamento       480394 non-null  object        
 5   Fecha de cierre    480394 non-null  datetime64[ns]
 6   Hora               480394 non-null  object        
 7   Tipo de cierre     480394 non-null  object        
 8   Serial             480394 non-null  object        
 9   Prioridad          480394 non-null  object        
 10  PCS                480394 non-null  object        
 11  PC                 480394 non-null  object        
 12  dicc_prioridades   479

* LO QUE ME CRUZO POR CLIENTE VALIDAR LOGICA

In [9]:
gestion_cc['Diferencia_Dias'] = (gestion_cc['Fecha de cierre'] - gestion_cc['FECHA_ENVIO IQE']).dt.days
gestion_cc['Rango_Dias'] = gestion_cc['Diferencia_Dias'].apply(lambda x: 'Si' if x >= -30 and x <= 60 else 'No')
gestion_cc=gestion_cc.drop_duplicates(subset="PCS")
print(gestion_cc.Rango_Dias.value_counts())
# Seleccionar las filas donde Rango_Dias es "No" y establecer las columnas especificadas en NaN
gestion_cc.loc[gestion_cc["Rango_Dias"] == "No", ["MACRO MOTIVO", "UNIDAD_DE_NEGOCIO", "FECHA_ENVIO IQE", "PRODUCTO", "TIPO BASE", "NOMBRE_BASE"]] = None
gestion_cc=gestion_cc.drop(columns=["Diferencia_Dias","Rango_Dias"],axis=1)
gestion_cc.reset_index(drop=True,inplace=True)
print(gestion_campo.info())

Rango_Dias
No    442494
Si     37900
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888257 entries, 0 to 888256
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   No. Contrato      888257 non-null  object        
 1   Cédula usuario    888257 non-null  object        
 2   Dirección         888257 non-null  object        
 3   Ciudad            888257 non-null  object        
 4   Departamento      888257 non-null  object        
 5   Fecha de cierre   888257 non-null  datetime64[ns]
 6   Hora              888257 non-null  object        
 7   Tipo de cierre    888257 non-null  object        
 8   Serial            888257 non-null  object        
 9   Prioridad         888257 non-null  object        
 10  PCS               888257 non-null  object        
 11  PC                888257 non-null  object        
 12  dicc_prioridades  885903 non-null  float64       
 

## GREAMOS GESTION CAMPO V3

In [10]:
gestion_campo=pd.concat([gestion_pc,gestion_cc],ignore_index=True)
gestion_campo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888257 entries, 0 to 888256
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   No. Contrato       888257 non-null  object        
 1   Cédula usuario     888257 non-null  object        
 2   Dirección          888257 non-null  object        
 3   Ciudad             888257 non-null  object        
 4   Departamento       888257 non-null  object        
 5   Fecha de cierre    888257 non-null  datetime64[ns]
 6   Hora               888257 non-null  object        
 7   Tipo de cierre     888257 non-null  object        
 8   Serial             888257 non-null  object        
 9   Prioridad          888257 non-null  object        
 10  PCS                888257 non-null  object        
 11  PC                 888257 non-null  object        
 12  dicc_prioridades   885903 non-null  float64       
 13  CRUCE              888257 non-null  object  

# CRUCE GESTION CAMPO VS SERIALES RECUPERADOS
El objetivo es traernos el nuevo serial recuperado 

In [11]:
gestion_campo["CRUCE"]="NO CRUZO"
gestion_campo.loc[gestion_campo["PC"].isin(seriales_recuperados["PC"]), "CRUCE" ] = "CRUZO"
gestion_campo= pd.merge(gestion_campo,
                     seriales_recuperados[["PC","Serial"]],
                     left_on="PC",right_on="PC",how="left",suffixes=('', '_RECUPERADO_WODEN'))
gestion_campo['PCS'] = gestion_campo['No. Contrato'].astype(str) + gestion_campo['Cédula usuario'].astype(str)+ gestion_campo['Serial_RECUPERADO_WODEN'].astype(str)    
gestion_campo=gestion_campo.drop_duplicates(subset="PCS")
# Agregar la columna "RECUPERADOR" basada en la condición de que "Serial_recuperado" no sea nula
gestion_campo['RECUPERADOR'] = gestion_campo['Serial_RECUPERADO_WODEN'].apply(lambda x: 'WODEN' if pd.notna(x) else '')
gestion_campo.reset_index(drop=True,inplace=True)
print(gestion_campo["CRUCE"].value_counts())
print(gestion_campo['RECUPERADOR'].value_counts(dropna=False))
print(gestion_campo.info())


CRUCE
CRUZO       305500
NO CRUZO    236791
Name: count, dtype: int64
RECUPERADOR
WODEN    305500
         236791
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542291 entries, 0 to 542290
Data columns (total 23 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   No. Contrato             542291 non-null  object        
 1   Cédula usuario           542291 non-null  object        
 2   Dirección                542291 non-null  object        
 3   Ciudad                   542291 non-null  object        
 4   Departamento             542291 non-null  object        
 5   Fecha de cierre          542291 non-null  datetime64[ns]
 6   Hora                     542291 non-null  object        
 7   Tipo de cierre           542291 non-null  object        
 8   Serial                   542291 non-null  object        
 9   Prioridad                542291 non-null  object        
 10  

In [12]:
gestion_campo.loc[gestion_campo["Cédula usuario"]=="1033338261"]

,No. Contrato,Cédula usuario,Dirección,Ciudad,Departamento,Fecha de cierre,Hora,Tipo de cierre,Serial,Prioridad,...,CRUCE,MACRO MOTIVO,UNIDAD_DE_NEGOCIO,FECHA_ENVIO IQE,PRODUCTO,TIPO BASE,NombreArchivo,NOMBRE_BASE,Serial_RECUPERADO_WODEN,RECUPERADOR
112342,1-65273746158112,1033338261,CL 85 # 90 - 62 IN 301,MEDELLIN,ANTIOQUIA,2024-03-23,11:16:41,RECUPERADO WODEN,14U4D9697507015,RECUPERADO,...,CRUZO,CARTERA,HOME,2024-03-18,INTERNET,BASE NACIONAL,BD432-031824,NaN,14U4D9697507015,WODEN
112343,1-65273746158112,1033338261,CL 85 # 90 - 62 IN 301,MEDELLIN,ANTIOQUIA,2024-03-23,11:16:41,RECUPERADO WODEN,14U4D9697507015,RECUPERADO,...,CRUZO,CARTERA,HOME,2024-03-18,INTERNET,BASE NACIONAL,BD432-031824,NaN,D7E000003DD2A7A,WODEN
112344,1-65273746158112,1033338261,CL 85 # 90 - 62 IN 301,MEDELLIN,ANTIOQUIA,2024-03-23,11:16:41,RECUPERADO WODEN,14U4D9697507015,RECUPERADO,...,CRUZO,CARTERA,HOME,2024-03-18,INTERNET,BASE NACIONAL,BD432-031824,NaN,D7E000006EB4EF0,WODEN
280020,1-65273746111652,1033338261,CL 85 # 90 - 62 IN 301,MEDELLIN,ANTIOQUIA,2024-03-19,12:55:34,CLIENTE NO ENCONTRADO,D7E000003DD2A7A,GESTIONADO,...,NO CRUZO,CARTERA,HOME,2024-03-18,TELEVISIÓN DIGITAL,BASE NACIONAL,BD432-031824,NaN,NaN,


# CRUCE GESTION CAMPO VS CANALES ALTERNOS

In [13]:
gestion_campo["CRUCE"]="NO CRUZO"
gestion_campo.loc[gestion_campo["Serial"].isin(canales_alternos["SERIALES"]), "CRUCE" ] = "CRUZO"
gestion_campo= pd.merge(gestion_campo,
                     canales_alternos[["SERIALES","FECHA","PROVEEDOR"]],
                     left_on="Serial",right_on="SERIALES",how="left",suffixes=('', '_canal_alterno'))
gestion_campo.drop(columns=["SERIALES"],inplace=True)
gestion_campo = gestion_campo.drop_duplicates(subset='PCS')
gestion_campo.reset_index(drop=True,inplace=True)
print(gestion_campo["CRUCE"].value_counts())
print(gestion_campo.info())

CRUCE
NO CRUZO    527505
CRUZO        14786
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542291 entries, 0 to 542290
Data columns (total 25 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   No. Contrato             542291 non-null  object        
 1   Cédula usuario           542291 non-null  object        
 2   Dirección                542291 non-null  object        
 3   Ciudad                   542291 non-null  object        
 4   Departamento             542291 non-null  object        
 5   Fecha de cierre          542291 non-null  datetime64[ns]
 6   Hora                     542291 non-null  object        
 7   Tipo de cierre           542291 non-null  object        
 8   Serial                   542291 non-null  object        
 9   Prioridad                542291 non-null  object        
 10  PCS                      542291 non-null  object        
 11  PC      

In [14]:
# Calcula la diferencia de días entre las dos fechas
gestion_campo['Diferencia_dias'] = (gestion_campo['FECHA'] - gestion_campo['Fecha de cierre']).dt.days
gestion_campo['Rango_Dias'] = gestion_campo['Diferencia_dias'].apply(lambda x: 'Si' if x >= -30 and x <= 60 else 'No')
print(gestion_campo.Rango_Dias.value_counts())
# Borrar la información en 'PROVEEDOR' y 'FECHA' cuando 'Rango_Dias' es 'No'
gestion_campo.loc[gestion_campo['Rango_Dias'] == 'No', ['PROVEEDOR', 'FECHA']] = None
gestion_campo['RECUPERADOR'] = np.where(gestion_campo['PROVEEDOR'].notnull(), gestion_campo['PROVEEDOR'], gestion_campo['RECUPERADOR'])
# Llenar los valores vacíos en 'RECUPERADOR' con 'OTROS CANALES'
gestion_campo['RECUPERADOR'].replace({"":'OTROS CANALES'},inplace=True)
gestion_campo.drop(columns=["Diferencia_dias","Rango_Dias"],inplace=True)
print(gestion_campo["RECUPERADOR"].value_counts(dropna=False))
print(gestion_campo.info())

Rango_Dias
No    530706
Si     11585
Name: count, dtype: int64
RECUPERADOR
WODEN            301503
OTROS CANALES    229203
FIELD              4736
Servientrega       3875
INTER              2974
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542291 entries, 0 to 542290
Data columns (total 25 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   No. Contrato             542291 non-null  object        
 1   Cédula usuario           542291 non-null  object        
 2   Dirección                542291 non-null  object        
 3   Ciudad                   542291 non-null  object        
 4   Departamento             542291 non-null  object        
 5   Fecha de cierre          542291 non-null  datetime64[ns]
 6   Hora                     542291 non-null  object        
 7   Tipo de cierre           542291 non-null  object        
 8   Serial                   542291 non-null  o

# CRUCE GESTION CAMPO VS CHURN


In [15]:
gestion_campo["CRUCE"]="NO CRUZO"
gestion_campo.loc[gestion_campo["Serial"].isin(churn["SERIAL_EQUIPO"]), "CRUCE" ] = "CRUZO"
gestion_campo=pd.merge(gestion_campo,  
                       churn[["SERIAL_EQUIPO","MARCA","REFERENCIA","TECNOL"]],
                       left_on="Serial",right_on="SERIAL_EQUIPO",how="left"
)
gestion_campo.drop(columns=["SERIAL_EQUIPO"],axis=1,inplace=True)
gestion_campo = gestion_campo.drop_duplicates(subset='PCS')
gestion_campo.reset_index(drop=True,inplace=True)
print(gestion_campo["CRUCE"].value_counts())
print(gestion_campo.info())

CRUCE
CRUZO       422406
NO CRUZO    119885
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542291 entries, 0 to 542290
Data columns (total 28 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   No. Contrato             542291 non-null  object        
 1   Cédula usuario           542291 non-null  object        
 2   Dirección                542291 non-null  object        
 3   Ciudad                   542291 non-null  object        
 4   Departamento             542291 non-null  object        
 5   Fecha de cierre          542291 non-null  datetime64[ns]
 6   Hora                     542291 non-null  object        
 7   Tipo de cierre           542291 non-null  object        
 8   Serial                   542291 non-null  object        
 9   Prioridad                542291 non-null  object        
 10  PCS                      542291 non-null  object        
 11  PC      

In [15]:
#gestion_campo.to_parquet('gestion_campo.parquet', index=False)

# SEGUNDA RAMA DEL CRUCE BASES ENVIADAS VS GESTION CAMPO

In [16]:
df_desconexiones = pd.read_parquet("C:/Users/oolea/Documents/01. Codigos PY/E2E- B2B/001. INFO OFICIAL ACTUALIZADA JUNIO/DATA E2E/DESCONEXIONES HISTORICO_CRUCE INVERSO.parquet")
df_desconexiones['PC'] = df_desconexiones['PEDIDO'].astype(str) + df_desconexiones['CLIENTE_ID'].astype(str)
df_desconexiones['PC']=df_desconexiones['PC'].str.upper()
desconexiones=df_desconexiones.copy()
desconexiones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502544 entries, 0 to 502543
Data columns (total 29 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   INTERFAZ                        502544 non-null  object
 1   PEDIDO                          502544 non-null  object
 2   IDENTIFICADOR                   502544 non-null  object
 3   IDUNICOSERVICIO                 502544 non-null  object
 4   IDSERVICIO_FACTURACION          502544 non-null  object
 5   FECHA_CORTE                     502544 non-null  object
 6   FECHA_CARGA                     502544 non-null  object
 7   FECHA_CUMPLIDO                  502544 non-null  object
 8   FECHA_INGRESO                   502544 non-null  object
 9   TIPO_TRANSACCION                502544 non-null  object
 10  PROCESO                         502544 non-null  object
 11  DEPARTAMENTO                    502544 non-null  object
 12  MUNICIPIO                     

In [17]:
gestion_campo_inversa=df1.copy()
bases_enviadas_inversa=df2.copy()

### BASES ENVIADAS
* Filtrar el DataFrame para incluir solo las filas donde la fecha de cierre sea igual o mayor a febrero 1 a abril 30
* Convertir la columna 'FECHA_ENVIO IQE' a tipo datetime
* Eliminar duplicados según la columna 'CC-S'

In [18]:
bases_enviadas_inversa = bases_enviadas_inversa[(bases_enviadas_inversa["FECHA_ENVIO IQE"] >= '2024-02-01')&(bases_enviadas_inversa["FECHA_ENVIO IQE"] <= '2024-04-30')]
bases_enviadas_inversa["FECHA_ENVIO IQE"]=pd.to_datetime(bases_enviadas_inversa["FECHA_ENVIO IQE"])
print("La fecha min de bases enviadas es: ",bases_enviadas_inversa["FECHA_ENVIO IQE"].min())
print("La fecha max de bases enviadas es: ",bases_enviadas_inversa["FECHA_ENVIO IQE"].max())
bases_enviadas_inversa.drop_duplicates(subset='CC-S', keep='first', inplace=True)
bases_enviadas_inversa.reset_index(drop=True,inplace=True)
bases_enviadas_inversa.info()

La fecha min de bases enviadas es:  2024-02-01 00:00:00
La fecha max de bases enviadas es:  2024-04-29 00:00:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383547 entries, 0 to 383546
Data columns (total 24 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   TIPO BASE          383547 non-null  object        
 1   PEDIDO             383547 non-null  object        
 2   IDENTIFICADOR_ID   383547 non-null  object        
 3   CLIENTE_ID         383547 non-null  object        
 4   INTERFAZ           383547 non-null  object        
 5   CIUDAD             383547 non-null  object        
 6   DEPARTAMENTO       383547 non-null  object        
 7   MARCA_ID           383547 non-null  object        
 8   REFERENCIA_ID      383547 non-null  object        
 9   ID_EQUIPO          383547 non-null  object        
 10  SERIAL_EQUIPO      383547 non-null  object        
 11  MAC_EQUIPO         383547 non-null  object  

### GESTION DE CAMPO
* Filtrar el DataFrame para incluir solo las filas donde la fecha de cierre sea igual o mayor a febrero 1 a abril 30
* Convertir la columna 'Fecha de cierre' y 'Hora' a tipo datetime
* Crear un diccionario de prioridades para el ordenamiento de 'Prioridad'
* Asignar prioridades numéricas a la columna 'dicc_prioridades'
* Ordenar el DataFrame por las columnas requeridas de mayor a menor dicc_prioridades,Fecha de cierre,Hora 
* Eliminar duplicados basados en la columna 'CC-S'

In [19]:
gestion_campo_inversa = gestion_campo_inversa[(gestion_campo_inversa["Fecha de cierre"] >= '2023-12-01')&(gestion_campo_inversa["Fecha de cierre"] <= '2024-06-04')]
gestion_campo_inversa['Fecha de cierre'] = pd.to_datetime(gestion_campo_inversa['Fecha de cierre'])
gestion_campo_inversa['Hora'] = pd.to_datetime(gestion_campo_inversa['Hora'], format='%H:%M:%S').dt.time
prioridades = {'RECUPERADO': 1, 'DESCARTADO': 2 ,'GESTIONADO': 3}
gestion_campo_inversa['dicc_prioridades'] = gestion_campo_inversa['Prioridad'].map(prioridades)
gestion_campo_inversa = gestion_campo_inversa.sort_values(by=['Prioridad','Fecha de cierre','Hora'], ascending=[False, False, False]).reset_index(drop=True)
gestion_campo_inversa = gestion_campo_inversa.drop_duplicates(subset='PCS')
gestion_campo_inversa.reset_index(drop=True,inplace=True)
print("La fecha min de gestion de campo es: ",gestion_campo_inversa["Fecha de cierre"].min())
print("La fecha max de gestion de campo es: ",gestion_campo_inversa["Fecha de cierre"].max())
gestion_campo_inversa.info()

La fecha min de gestion de campo es:  2024-01-03 00:00:00
La fecha max de gestion de campo es:  2024-06-04 00:00:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888257 entries, 0 to 888256
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   No. Contrato      888257 non-null  object        
 1   Cédula usuario    888257 non-null  object        
 2   Dirección         888257 non-null  object        
 3   Ciudad            888257 non-null  object        
 4   Departamento      888257 non-null  object        
 5   Fecha de cierre   888257 non-null  datetime64[ns]
 6   Hora              888257 non-null  object        
 7   Tipo de cierre    888257 non-null  object        
 8   Serial            888257 non-null  object        
 9   Prioridad         888257 non-null  object        
 10  PCS               888257 non-null  object        
 11  PC                888257 non-null  object        
 12

# CRUCE DESCONEXIONES VS BASES ENVIADAS

In [ ]:
#creamos una columa llamada cruce con el valor fijo de NO CRUZO
#desconexiones["CRUCE"]="NO CRUZO"
#desconexiones.loc[desconexiones["PC"].isin(bases_enviadas_inversa["PC"]), "CRUCE" ] = "CRUZO PC"
#print(desconexiones["CRUCE"].value_counts())

In [ ]:
#desconexiones_pc=desconexiones[desconexiones["CRUCE"]=="CRUZO PC"]
#desconexiones_pc.info()

In [ ]:
#desconexiones_nocruzo=desconexiones[desconexiones["CRUCE"]=="NO CRUZO"]
#desconexiones_nocruzo.info()

In [ ]:
#desconexiones_pc_prueba= pd.merge(desconexiones_pc,
#                     bases_enviadas_inversa[["PC"]],
#                     left_on="PC",right_on="PC",how="left")
#desconexiones_pc_prueba=desconexiones_pc_prueba.drop_duplicates(subset="PC")
#desconexiones_pc_prueba.reset_index(drop=True,inplace=True)
#print(desconexiones_pc_prueba.info())

# CRUCE BASES_ENVIADAS VS GESTION CAMPO


In [20]:
#creamos una columa llamada cruce con el valor fijo de NO CRUZO
bases_enviadas_inversa["CRUCE"]="NO CRUZO"
bases_enviadas_inversa.loc[bases_enviadas_inversa["PC"].isin(gestion_campo_inversa["PC"]), "CRUCE" ] = "CRUZO PC"
print(bases_enviadas_inversa["CRUCE"].value_counts())
bases_enviadas_pc= pd.merge(bases_enviadas_inversa,
                     gestion_campo_inversa[["PC","Fecha de cierre","Tipo de cierre"]],
                     left_on="PC",right_on="PC",how="left")
bases_enviadas_pc=bases_enviadas_pc.drop_duplicates(subset="CC-S")
bases_enviadas_pc.reset_index(drop=True,inplace=True)
print(bases_enviadas_pc.info())

CRUCE
CRUZO PC    292888
NO CRUZO     90659
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383547 entries, 0 to 383546
Data columns (total 27 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   TIPO BASE          383547 non-null  object        
 1   PEDIDO             383547 non-null  object        
 2   IDENTIFICADOR_ID   383547 non-null  object        
 3   CLIENTE_ID         383547 non-null  object        
 4   INTERFAZ           383547 non-null  object        
 5   CIUDAD             383547 non-null  object        
 6   DEPARTAMENTO       383547 non-null  object        
 7   MARCA_ID           383547 non-null  object        
 8   REFERENCIA_ID      383547 non-null  object        
 9   ID_EQUIPO          383547 non-null  object        
 10  SERIAL_EQUIPO      383547 non-null  object        
 11  MAC_EQUIPO         383547 non-null  object        
 12  PRODUCTO           383547 non-

In [21]:
be_cruzo_pc=bases_enviadas_pc.loc[bases_enviadas_pc["CRUCE"]=="CRUZO PC"]
be_cruzo_pc["PROVEEDOR"]="WODEN"
be_cruzo_pc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 292888 entries, 0 to 383546
Data columns (total 28 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   TIPO BASE          292888 non-null  object        
 1   PEDIDO             292888 non-null  object        
 2   IDENTIFICADOR_ID   292888 non-null  object        
 3   CLIENTE_ID         292888 non-null  object        
 4   INTERFAZ           292888 non-null  object        
 5   CIUDAD             292888 non-null  object        
 6   DEPARTAMENTO       292888 non-null  object        
 7   MARCA_ID           292888 non-null  object        
 8   REFERENCIA_ID      292888 non-null  object        
 9   ID_EQUIPO          292888 non-null  object        
 10  SERIAL_EQUIPO      292888 non-null  object        
 11  MAC_EQUIPO         292888 non-null  object        
 12  PRODUCTO           292888 non-null  object        
 13  TECNOLOGÍA         292888 non-null  object       

In [22]:
be_nocruzo_pc=bases_enviadas_inversa.loc[bases_enviadas_inversa["CRUCE"]=="NO CRUZO"]
be_nocruzo_pc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 90659 entries, 30 to 383536
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          90659 non-null  object        
 1   PEDIDO             90659 non-null  object        
 2   IDENTIFICADOR_ID   90659 non-null  object        
 3   CLIENTE_ID         90659 non-null  object        
 4   INTERFAZ           90659 non-null  object        
 5   CIUDAD             90659 non-null  object        
 6   DEPARTAMENTO       90659 non-null  object        
 7   MARCA_ID           90659 non-null  object        
 8   REFERENCIA_ID      90659 non-null  object        
 9   ID_EQUIPO          90659 non-null  object        
 10  SERIAL_EQUIPO      90659 non-null  object        
 11  MAC_EQUIPO         90659 non-null  object        
 12  PRODUCTO           90659 non-null  object        
 13  TECNOLOGÍA         90659 non-null  object        
 14  UNIDAD_DE

In [23]:
#creamos una columa llamada cruce con el valor fijo de NO CRUZO
be_nocruzo_pc["CRUCE"]="NO CRUZO"
be_nocruzo_pc.loc[be_nocruzo_pc["CLIENTE_ID"].isin(gestion_campo_inversa["Cédula usuario"]), "CRUCE" ] = "CRUZO CC"
be_nocruzo_pc["CRUCE"].value_counts()

CRUCE
CRUZO CC    75591
NO CRUZO    15068
Name: count, dtype: int64

In [24]:
be_cruzo_cc=be_nocruzo_pc.loc[be_nocruzo_pc["CRUCE"]=="CRUZO CC"]
be_cruzo_cc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 75591 entries, 46 to 383536
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          75591 non-null  object        
 1   PEDIDO             75591 non-null  object        
 2   IDENTIFICADOR_ID   75591 non-null  object        
 3   CLIENTE_ID         75591 non-null  object        
 4   INTERFAZ           75591 non-null  object        
 5   CIUDAD             75591 non-null  object        
 6   DEPARTAMENTO       75591 non-null  object        
 7   MARCA_ID           75591 non-null  object        
 8   REFERENCIA_ID      75591 non-null  object        
 9   ID_EQUIPO          75591 non-null  object        
 10  SERIAL_EQUIPO      75591 non-null  object        
 11  MAC_EQUIPO         75591 non-null  object        
 12  PRODUCTO           75591 non-null  object        
 13  TECNOLOGÍA         75591 non-null  object        
 14  UNIDAD_DE

In [25]:
be_cruzo_cc= pd.merge(be_cruzo_cc,
                     gestion_campo_inversa[["Cédula usuario",'Fecha de cierre', 'Tipo de cierre']],
                     left_on="CLIENTE_ID",right_on="Cédula usuario",how="left",suffixes=('', '_gestion_campo'))
be_cruzo_cc=be_cruzo_cc.drop_duplicates(subset="CC-S")
be_cruzo_cc.reset_index(drop=True,inplace=True)
print(be_cruzo_cc.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75591 entries, 0 to 75590
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          75591 non-null  object        
 1   PEDIDO             75591 non-null  object        
 2   IDENTIFICADOR_ID   75591 non-null  object        
 3   CLIENTE_ID         75591 non-null  object        
 4   INTERFAZ           75591 non-null  object        
 5   CIUDAD             75591 non-null  object        
 6   DEPARTAMENTO       75591 non-null  object        
 7   MARCA_ID           75591 non-null  object        
 8   REFERENCIA_ID      75591 non-null  object        
 9   ID_EQUIPO          75591 non-null  object        
 10  SERIAL_EQUIPO      75591 non-null  object        
 11  MAC_EQUIPO         75591 non-null  object        
 12  PRODUCTO           75591 non-null  object        
 13  TECNOLOGÍA         75591 non-null  object        
 14  UNIDAD

In [26]:
be_cruzo_cc['Diferencia_Dias'] = (be_cruzo_cc['Fecha de cierre'] - be_cruzo_cc['FECHA_ENVIO IQE']).dt.days
be_cruzo_cc['Rango_Dias'] = be_cruzo_cc['Diferencia_Dias'].apply(lambda x: 'Si' if x >= -30 and x <= 60 else 'No')
be_cruzo_cc=be_cruzo_cc.drop_duplicates(subset="CC-S")
print(be_cruzo_cc.Rango_Dias.value_counts())

Rango_Dias
Si    72054
No     3537
Name: count, dtype: int64


In [27]:
be_cruzo_cc_logicasi=be_cruzo_cc[be_cruzo_cc['Rango_Dias']=="Si"]
be_cruzo_cc_logicasi.drop(["Cédula usuario","Diferencia_Dias","Rango_Dias"],axis=1,inplace=True)
be_cruzo_cc_logicasi["PROVEEDOR"]="WODEN"
be_cruzo_cc_logicasi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 72054 entries, 0 to 75589
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          72054 non-null  object        
 1   PEDIDO             72054 non-null  object        
 2   IDENTIFICADOR_ID   72054 non-null  object        
 3   CLIENTE_ID         72054 non-null  object        
 4   INTERFAZ           72054 non-null  object        
 5   CIUDAD             72054 non-null  object        
 6   DEPARTAMENTO       72054 non-null  object        
 7   MARCA_ID           72054 non-null  object        
 8   REFERENCIA_ID      72054 non-null  object        
 9   ID_EQUIPO          72054 non-null  object        
 10  SERIAL_EQUIPO      72054 non-null  object        
 11  MAC_EQUIPO         72054 non-null  object        
 12  PRODUCTO           72054 non-null  object        
 13  TECNOLOGÍA         72054 non-null  object        
 14  UNIDAD_DE_N

In [28]:
be_nocruzo_cc=be_nocruzo_pc.loc[be_nocruzo_pc["CRUCE"]=="NO CRUZO"]
be_nocruzo_cc_logicano=be_cruzo_cc.loc[be_cruzo_cc["Rango_Dias"]=="No"]

In [29]:
concatenado_nocruzo=pd.concat([be_nocruzo_cc,be_nocruzo_cc_logicano],ignore_index=True)
concatenado_nocruzo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18605 entries, 0 to 18604
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          18605 non-null  object        
 1   PEDIDO             18605 non-null  object        
 2   IDENTIFICADOR_ID   18605 non-null  object        
 3   CLIENTE_ID         18605 non-null  object        
 4   INTERFAZ           18605 non-null  object        
 5   CIUDAD             18605 non-null  object        
 6   DEPARTAMENTO       18605 non-null  object        
 7   MARCA_ID           18605 non-null  object        
 8   REFERENCIA_ID      18605 non-null  object        
 9   ID_EQUIPO          18605 non-null  object        
 10  SERIAL_EQUIPO      18605 non-null  object        
 11  MAC_EQUIPO         18605 non-null  object        
 12  PRODUCTO           18605 non-null  object        
 13  TECNOLOGÍA         18605 non-null  object        
 14  UNIDAD

In [30]:
# Seleccionar las filas donde Rango_Dias es "No" y establecer las columnas especificadas en NaN
concatenado_nocruzo.loc[concatenado_nocruzo["Rango_Dias"] == "No", ['Cédula usuario','Fecha de cierre', 'Tipo de cierre', 'Diferencia_Dias','Rango_Dias']] = None
concatenado_nocruzo.drop(["Diferencia_Dias","Rango_Dias","Cédula usuario"],axis=1,inplace=True)
concatenado_nocruzo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18605 entries, 0 to 18604
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          18605 non-null  object        
 1   PEDIDO             18605 non-null  object        
 2   IDENTIFICADOR_ID   18605 non-null  object        
 3   CLIENTE_ID         18605 non-null  object        
 4   INTERFAZ           18605 non-null  object        
 5   CIUDAD             18605 non-null  object        
 6   DEPARTAMENTO       18605 non-null  object        
 7   MARCA_ID           18605 non-null  object        
 8   REFERENCIA_ID      18605 non-null  object        
 9   ID_EQUIPO          18605 non-null  object        
 10  SERIAL_EQUIPO      18605 non-null  object        
 11  MAC_EQUIPO         18605 non-null  object        
 12  PRODUCTO           18605 non-null  object        
 13  TECNOLOGÍA         18605 non-null  object        
 14  UNIDAD

# CANAL ALTERNO CRUCE INVERSO

In [31]:
canal_alterno_cruce_inverso=df4.copy()
canal_alterno_cruce_inverso.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 527287 entries, 0 to 527286
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   GUIA        291456 non-null  object
 1   LLAVE       527255 non-null  object
 2   CLIENTE_ID  481867 non-null  object
 3   SERIALES    527214 non-null  object
 4   FECHA       527287 non-null  object
 5   PROVEEDOR   527287 non-null  object
dtypes: object(6)
memory usage: 24.1+ MB


In [32]:
canal_alterno_cruce_inverso['FECHA'] = pd.to_datetime(canal_alterno_cruce_inverso['FECHA'], format='%d/%m/%Y')
# Filtrar los datos según el rango de fechas
canal_alterno_cruce_inverso = canal_alterno_cruce_inverso[(canal_alterno_cruce_inverso['FECHA'] >= '2024-01-01') & (canal_alterno_cruce_inverso['FECHA'] <= '2024-06-30')]
canal_alterno_cruce_inverso = canal_alterno_cruce_inverso.sort_values(by=['FECHA'], ascending=False)
canal_alterno_cruce_inverso = canal_alterno_cruce_inverso.drop_duplicates()
canal_alterno_cruce_inverso.reset_index(drop=True,inplace=True)
print("La fecha min de seriales_recuperados es: ",canal_alterno_cruce_inverso["FECHA"].min())
print("La fecha max de seriales_recuperados es: ",canal_alterno_cruce_inverso["FECHA"].max())
canal_alterno_cruce_inverso.info()

La fecha min de seriales_recuperados es:  2024-01-03 00:00:00
La fecha max de seriales_recuperados es:  2024-06-10 00:00:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316496 entries, 0 to 316495
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   GUIA        85708 non-null   object        
 1   LLAVE       316490 non-null  object        
 2   CLIENTE_ID  300810 non-null  object        
 3   SERIALES    316483 non-null  object        
 4   FECHA       316496 non-null  datetime64[ns]
 5   PROVEEDOR   316496 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 14.5+ MB


In [33]:
#creamos una columa llamada cruce con el valor fijo de NO CRUZO
concatenado_nocruzo["CRUCE"]="NO CRUZO"
concatenado_nocruzo.loc[concatenado_nocruzo["CC-S"].isin(canal_alterno_cruce_inverso["LLAVE"]), "CRUCE" ] = "CRUZO"
concatenado_nocruzo["CRUCE"].value_counts(dropna=False)

CRUCE
NO CRUZO    17756
CRUZO         849
Name: count, dtype: int64

In [34]:
be_ca_cruzo_ccs=concatenado_nocruzo[concatenado_nocruzo["CRUCE"]=="CRUZO"]
be_ca_cruzo_ccs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 849 entries, 2 to 18452
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          849 non-null    object        
 1   PEDIDO             849 non-null    object        
 2   IDENTIFICADOR_ID   849 non-null    object        
 3   CLIENTE_ID         849 non-null    object        
 4   INTERFAZ           849 non-null    object        
 5   CIUDAD             849 non-null    object        
 6   DEPARTAMENTO       849 non-null    object        
 7   MARCA_ID           849 non-null    object        
 8   REFERENCIA_ID      849 non-null    object        
 9   ID_EQUIPO          849 non-null    object        
 10  SERIAL_EQUIPO      849 non-null    object        
 11  MAC_EQUIPO         849 non-null    object        
 12  PRODUCTO           849 non-null    object        
 13  TECNOLOGÍA         849 non-null    object        
 14  UNIDAD_DE_NEG

In [35]:
be_ca_cruzo_ccs= pd.merge(be_ca_cruzo_ccs,
                     canal_alterno_cruce_inverso[["LLAVE","FECHA",'PROVEEDOR']],
                     left_on="CC-S",right_on="LLAVE",how="left",suffixes=('', '_canal_alterno'))
be_ca_cruzo_ccs["Tipo de cierre"]="RECUPERADO"
be_ca_cruzo_ccs["Fecha de cierre"]=be_ca_cruzo_ccs["FECHA_canal_alterno"].copy()
be_ca_cruzo_ccs.drop(["LLAVE","FECHA_canal_alterno"],axis=1,inplace=True)
#concatenado_llave_canal_alterno=concatenado_llave_canal_alterno.drop_duplicates(subset="CC-S")
be_ca_cruzo_ccs["Tipo de cierre"]="RECUPERADO"
be_ca_cruzo_ccs.reset_index(drop=True,inplace=True)
print(be_ca_cruzo_ccs.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 849 entries, 0 to 848
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          849 non-null    object        
 1   PEDIDO             849 non-null    object        
 2   IDENTIFICADOR_ID   849 non-null    object        
 3   CLIENTE_ID         849 non-null    object        
 4   INTERFAZ           849 non-null    object        
 5   CIUDAD             849 non-null    object        
 6   DEPARTAMENTO       849 non-null    object        
 7   MARCA_ID           849 non-null    object        
 8   REFERENCIA_ID      849 non-null    object        
 9   ID_EQUIPO          849 non-null    object        
 10  SERIAL_EQUIPO      849 non-null    object        
 11  MAC_EQUIPO         849 non-null    object        
 12  PRODUCTO           849 non-null    object        
 13  TECNOLOGÍA         849 non-null    object        
 14  UNIDAD_DE_

In [36]:
be_ca_nocruzo_ccs=concatenado_nocruzo[concatenado_nocruzo["CRUCE"]=="NO CRUZO"]
be_ca_nocruzo_ccs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17756 entries, 0 to 18604
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          17756 non-null  object        
 1   PEDIDO             17756 non-null  object        
 2   IDENTIFICADOR_ID   17756 non-null  object        
 3   CLIENTE_ID         17756 non-null  object        
 4   INTERFAZ           17756 non-null  object        
 5   CIUDAD             17756 non-null  object        
 6   DEPARTAMENTO       17756 non-null  object        
 7   MARCA_ID           17756 non-null  object        
 8   REFERENCIA_ID      17756 non-null  object        
 9   ID_EQUIPO          17756 non-null  object        
 10  SERIAL_EQUIPO      17756 non-null  object        
 11  MAC_EQUIPO         17756 non-null  object        
 12  PRODUCTO           17756 non-null  object        
 13  TECNOLOGÍA         17756 non-null  object        
 14  UNIDAD_DE_N

In [37]:
#creamos una columa llamada cruce con el valor fijo de NO CRUZO
be_ca_nocruzo_ccs["CRUCE"]="NO CRUZO"
be_ca_nocruzo_ccs.loc[be_ca_nocruzo_ccs["CLIENTE_ID"].isin(canal_alterno_cruce_inverso["CLIENTE_ID"]), "CRUCE" ] = "CRUZO"
be_ca_nocruzo_ccs["CRUCE"].value_counts(dropna=False)

CRUCE
NO CRUZO    16167
CRUZO        1589
Name: count, dtype: int64

In [38]:
be_ca_cruzo_cc=be_ca_nocruzo_ccs[be_ca_nocruzo_ccs["CRUCE"]=="CRUZO"]
be_ca_cruzo_cc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1589 entries, 54 to 18598
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          1589 non-null   object        
 1   PEDIDO             1589 non-null   object        
 2   IDENTIFICADOR_ID   1589 non-null   object        
 3   CLIENTE_ID         1589 non-null   object        
 4   INTERFAZ           1589 non-null   object        
 5   CIUDAD             1589 non-null   object        
 6   DEPARTAMENTO       1589 non-null   object        
 7   MARCA_ID           1589 non-null   object        
 8   REFERENCIA_ID      1589 non-null   object        
 9   ID_EQUIPO          1589 non-null   object        
 10  SERIAL_EQUIPO      1589 non-null   object        
 11  MAC_EQUIPO         1589 non-null   object        
 12  PRODUCTO           1589 non-null   object        
 13  TECNOLOGÍA         1589 non-null   object        
 14  UNIDAD_DE_N

In [39]:
be_ca_cruzo_cc= pd.merge(be_ca_cruzo_cc,
                     canal_alterno_cruce_inverso[["CLIENTE_ID","FECHA","PROVEEDOR"]],
                     left_on="CLIENTE_ID",right_on="CLIENTE_ID",how="inner",suffixes=('', '_canal_alterno'))
be_ca_cruzo_cc=be_ca_cruzo_cc.drop_duplicates(subset="CC-S")
be_ca_cruzo_cc["Tipo de cierre"]="RECUPERADO"
be_ca_cruzo_cc["Fecha de cierre"]=be_ca_cruzo_cc["FECHA_canal_alterno"].copy()
be_ca_cruzo_cc.reset_index(drop=True,inplace=True)
print(be_ca_cruzo_cc.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1589 entries, 0 to 1588
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   TIPO BASE            1589 non-null   object        
 1   PEDIDO               1589 non-null   object        
 2   IDENTIFICADOR_ID     1589 non-null   object        
 3   CLIENTE_ID           1589 non-null   object        
 4   INTERFAZ             1589 non-null   object        
 5   CIUDAD               1589 non-null   object        
 6   DEPARTAMENTO         1589 non-null   object        
 7   MARCA_ID             1589 non-null   object        
 8   REFERENCIA_ID        1589 non-null   object        
 9   ID_EQUIPO            1589 non-null   object        
 10  SERIAL_EQUIPO        1589 non-null   object        
 11  MAC_EQUIPO           1589 non-null   object        
 12  PRODUCTO             1589 non-null   object        
 13  TECNOLOGÍA           1589 non-nul

In [40]:
be_ca_cruzo_cc['FECHA'] = pd.to_datetime(canal_alterno_cruce_inverso['FECHA'], format='%d/%m/%Y')
be_ca_cruzo_cc['Diferencia_Dias'] = (be_ca_cruzo_cc['FECHA'] - be_ca_cruzo_cc['FECHA_ENVIO IQE']).dt.days
be_ca_cruzo_cc['Rango_Dias'] = be_ca_cruzo_cc['Diferencia_Dias'].apply(lambda x: 'Si' if x >= -30 and x <= 60 else 'No')
be_ca_cruzo_cc=be_ca_cruzo_cc.drop_duplicates(subset="CC-S")
print(be_ca_cruzo_cc.Rango_Dias.value_counts())

Rango_Dias
No    931
Si    658
Name: count, dtype: int64


In [41]:
be_ca_cruzo_cc_logicasi=be_ca_cruzo_cc[be_ca_cruzo_cc['Rango_Dias']=="Si"]
be_ca_cruzo_cc_logicasi.drop(["FECHA_canal_alterno","Diferencia_Dias","Rango_Dias"],axis=1,inplace=True)
be_ca_cruzo_cc_logicasi["Tipo de cierre"]="RECUPERADO"
be_ca_cruzo_cc_logicasi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 658 entries, 109 to 1588
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          658 non-null    object        
 1   PEDIDO             658 non-null    object        
 2   IDENTIFICADOR_ID   658 non-null    object        
 3   CLIENTE_ID         658 non-null    object        
 4   INTERFAZ           658 non-null    object        
 5   CIUDAD             658 non-null    object        
 6   DEPARTAMENTO       658 non-null    object        
 7   MARCA_ID           658 non-null    object        
 8   REFERENCIA_ID      658 non-null    object        
 9   ID_EQUIPO          658 non-null    object        
 10  SERIAL_EQUIPO      658 non-null    object        
 11  MAC_EQUIPO         658 non-null    object        
 12  PRODUCTO           658 non-null    object        
 13  TECNOLOGÍA         658 non-null    object        
 14  UNIDAD_DE_NE

In [42]:
novedad1=be_ca_nocruzo_ccs[be_ca_nocruzo_ccs["CRUCE"]=="NO CRUZO"]
novedad2=be_ca_cruzo_cc[be_ca_cruzo_cc["Rango_Dias"]=="No"]

In [43]:
concatenado_nocruzo=pd.concat([novedad1,novedad2],ignore_index=True)
concatenado_nocruzo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17098 entries, 0 to 17097
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   TIPO BASE            17098 non-null  object        
 1   PEDIDO               17098 non-null  object        
 2   IDENTIFICADOR_ID     17098 non-null  object        
 3   CLIENTE_ID           17098 non-null  object        
 4   INTERFAZ             17098 non-null  object        
 5   CIUDAD               17098 non-null  object        
 6   DEPARTAMENTO         17098 non-null  object        
 7   MARCA_ID             17098 non-null  object        
 8   REFERENCIA_ID        17098 non-null  object        
 9   ID_EQUIPO            17098 non-null  object        
 10  SERIAL_EQUIPO        17098 non-null  object        
 11  MAC_EQUIPO           17098 non-null  object        
 12  PRODUCTO             17098 non-null  object        
 13  TECNOLOGÍA           17098 non-

In [44]:
# Seleccionar las filas donde Rango_Dias es "No" y establecer las columnas especificadas en NaN
concatenado_nocruzo.loc[concatenado_nocruzo["Rango_Dias"] == "No", ["FECHA_canal_alterno","PROVEEDOR","Tipo de cierre","Fecha de cierre"]] = None
concatenado_nocruzo.drop(["Diferencia_Dias","Rango_Dias","FECHA_canal_alterno","PROVEEDOR"],axis=1,inplace=True)
concatenado_nocruzo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17098 entries, 0 to 17097
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          17098 non-null  object        
 1   PEDIDO             17098 non-null  object        
 2   IDENTIFICADOR_ID   17098 non-null  object        
 3   CLIENTE_ID         17098 non-null  object        
 4   INTERFAZ           17098 non-null  object        
 5   CIUDAD             17098 non-null  object        
 6   DEPARTAMENTO       17098 non-null  object        
 7   MARCA_ID           17098 non-null  object        
 8   REFERENCIA_ID      17098 non-null  object        
 9   ID_EQUIPO          17098 non-null  object        
 10  SERIAL_EQUIPO      17098 non-null  object        
 11  MAC_EQUIPO         17098 non-null  object        
 12  PRODUCTO           17098 non-null  object        
 13  TECNOLOGÍA         17098 non-null  object        
 14  UNIDAD

In [45]:
bases_enviadas_cruzo=pd.concat([be_cruzo_pc,be_cruzo_cc_logicasi,be_ca_cruzo_ccs,be_ca_cruzo_cc_logicasi],ignore_index=True)
print(bases_enviadas_cruzo.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366449 entries, 0 to 366448
Data columns (total 28 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   TIPO BASE          366449 non-null  object        
 1   PEDIDO             366449 non-null  object        
 2   IDENTIFICADOR_ID   366449 non-null  object        
 3   CLIENTE_ID         366449 non-null  object        
 4   INTERFAZ           366449 non-null  object        
 5   CIUDAD             366449 non-null  object        
 6   DEPARTAMENTO       366449 non-null  object        
 7   MARCA_ID           366449 non-null  object        
 8   REFERENCIA_ID      366449 non-null  object        
 9   ID_EQUIPO          366449 non-null  object        
 10  SERIAL_EQUIPO      366449 non-null  object        
 11  MAC_EQUIPO         366449 non-null  object        
 12  PRODUCTO           366449 non-null  object        
 13  TECNOLOGÍA         366449 non-null  object  

In [46]:
bases_enviadas_cruzo["PROVEEDOR"].value_counts()

PROVEEDOR
WODEN           364942
INTER              887
Servientrega       447
FIELD              173
Name: count, dtype: int64

In [47]:
provedor_woden=bases_enviadas_cruzo[bases_enviadas_cruzo["PROVEEDOR"]=="WODEN"]
provedor_woden.info()

<class 'pandas.core.frame.DataFrame'>
Index: 364942 entries, 0 to 364941
Data columns (total 28 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   TIPO BASE          364942 non-null  object        
 1   PEDIDO             364942 non-null  object        
 2   IDENTIFICADOR_ID   364942 non-null  object        
 3   CLIENTE_ID         364942 non-null  object        
 4   INTERFAZ           364942 non-null  object        
 5   CIUDAD             364942 non-null  object        
 6   DEPARTAMENTO       364942 non-null  object        
 7   MARCA_ID           364942 non-null  object        
 8   REFERENCIA_ID      364942 non-null  object        
 9   ID_EQUIPO          364942 non-null  object        
 10  SERIAL_EQUIPO      364942 non-null  object        
 11  MAC_EQUIPO         364942 non-null  object        
 12  PRODUCTO           364942 non-null  object        
 13  TECNOLOGÍA         364942 non-null  object       

In [48]:
provedor_no_woden=bases_enviadas_cruzo[bases_enviadas_cruzo["PROVEEDOR"]!="WODEN"]
provedor_no_woden.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1507 entries, 364942 to 366448
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          1507 non-null   object        
 1   PEDIDO             1507 non-null   object        
 2   IDENTIFICADOR_ID   1507 non-null   object        
 3   CLIENTE_ID         1507 non-null   object        
 4   INTERFAZ           1507 non-null   object        
 5   CIUDAD             1507 non-null   object        
 6   DEPARTAMENTO       1507 non-null   object        
 7   MARCA_ID           1507 non-null   object        
 8   REFERENCIA_ID      1507 non-null   object        
 9   ID_EQUIPO          1507 non-null   object        
 10  SERIAL_EQUIPO      1507 non-null   object        
 11  MAC_EQUIPO         1507 non-null   object        
 12  PRODUCTO           1507 non-null   object        
 13  TECNOLOGÍA         1507 non-null   object        
 14  UNIDAD

In [49]:
# Filtrar las filas que no tienen las tipificaciones "RECUPERADO WODEN" y "ENTREGO A WODEN"
filtrado2 = bases_enviadas_cruzo[bases_enviadas_cruzo["Tipo de cierre"].isin(["RECUPERADO WODEN", "ENTREGO A WODEN"])]
filtrado2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 251798 entries, 0 to 364940
Data columns (total 28 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   TIPO BASE          251798 non-null  object        
 1   PEDIDO             251798 non-null  object        
 2   IDENTIFICADOR_ID   251798 non-null  object        
 3   CLIENTE_ID         251798 non-null  object        
 4   INTERFAZ           251798 non-null  object        
 5   CIUDAD             251798 non-null  object        
 6   DEPARTAMENTO       251798 non-null  object        
 7   MARCA_ID           251798 non-null  object        
 8   REFERENCIA_ID      251798 non-null  object        
 9   ID_EQUIPO          251798 non-null  object        
 10  SERIAL_EQUIPO      251798 non-null  object        
 11  MAC_EQUIPO         251798 non-null  object        
 12  PRODUCTO           251798 non-null  object        
 13  TECNOLOGÍA         251798 non-null  object       

In [50]:
# Filtrar las filas que no tienen las tipificaciones "RECUPERADO WODEN" y "ENTREGO A WODEN"
filtrado = provedor_woden[~provedor_woden["Tipo de cierre"].isin(["RECUPERADO WODEN", "ENTREGO A WODEN"])]
# Verificar los resultados
print(filtrado["Tipo de cierre"].value_counts(dropna=False))
print(len(filtrado))

Tipo de cierre
CLIENTE NO ENCONTRADO                           42221
CLIENTE NO VIVE EN DIRECCIÓN                    26134
CLIENTE ENTREGO A TERCEROS                      11037
VOLVER A VISITAR                                10206
DIRECCION DEFICIENTE                             4201
DIRECCIÓN NO EXISTE                              3320
CLIENTE ENTREGO A SERVIENTREGA                   2024
CLIENTE ENTREGO A TERCEROS (INSTALADOR)          1922
CLIENTE RECONECTADO                              1552
CLIENTE INCUMPLE CITA                            1467
CLIENTE ENTREGO A TERCEROS (INTERRAPIDISIMO)     1345
SERVICIO ACTIVO                                  1054
*SELECCIONAR*                                    1040
CLIENTE NO ENTREGA EQUIPOS                        910
ORDEN PÚBLICO                                     867
CLIENTE NO CUENTA CON LOS EQUIPOS                 759
FRAUDE                                            729
CLIENTE DE VIAJE                                  679
CAMBIO DE DIR

In [51]:
# Realizar el cruce
merged_df = filtrado.merge(canal_alterno_cruce_inverso[['LLAVE', 'PROVEEDOR']],left_on="CC-S",right_on="LLAVE", how='left', suffixes=('', '_nuevo'))

# Actualizar la columna 'PROVEEDOR'
merged_df['PROVEEDOR'] = merged_df['PROVEEDOR_nuevo'].combine_first(merged_df['PROVEEDOR'])

# Eliminar la columna 'PROVEEDOR_nuevo' ya que no es necesaria después de la actualización
merged_df = merged_df.drop(columns=['PROVEEDOR_nuevo'])
merged_df['PROVEEDOR'].value_counts(dropna=False)

PROVEEDOR
WODEN           103993
Servientrega      6877
FIELD             1249
INTER             1082
Name: count, dtype: int64

In [52]:
merged_df_woden=merged_df[merged_df['PROVEEDOR']=="WODEN"]
merged_df_woden.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103993 entries, 0 to 113200
Data columns (total 29 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   TIPO BASE          103993 non-null  object        
 1   PEDIDO             103993 non-null  object        
 2   IDENTIFICADOR_ID   103993 non-null  object        
 3   CLIENTE_ID         103993 non-null  object        
 4   INTERFAZ           103993 non-null  object        
 5   CIUDAD             103993 non-null  object        
 6   DEPARTAMENTO       103993 non-null  object        
 7   MARCA_ID           103993 non-null  object        
 8   REFERENCIA_ID      103993 non-null  object        
 9   ID_EQUIPO          103993 non-null  object        
 10  SERIAL_EQUIPO      103993 non-null  object        
 11  MAC_EQUIPO         103993 non-null  object        
 12  PRODUCTO           103993 non-null  object        
 13  TECNOLOGÍA         103993 non-null  object       

In [53]:
merged_df_no_woden=merged_df[merged_df['PROVEEDOR']!="WODEN"]
merged_df_no_woden.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9208 entries, 3 to 113190
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          9208 non-null   object        
 1   PEDIDO             9208 non-null   object        
 2   IDENTIFICADOR_ID   9208 non-null   object        
 3   CLIENTE_ID         9208 non-null   object        
 4   INTERFAZ           9208 non-null   object        
 5   CIUDAD             9208 non-null   object        
 6   DEPARTAMENTO       9208 non-null   object        
 7   MARCA_ID           9208 non-null   object        
 8   REFERENCIA_ID      9208 non-null   object        
 9   ID_EQUIPO          9208 non-null   object        
 10  SERIAL_EQUIPO      9208 non-null   object        
 11  MAC_EQUIPO         9208 non-null   object        
 12  PRODUCTO           9208 non-null   object        
 13  TECNOLOGÍA         9208 non-null   object        
 14  UNIDAD_DE_N

In [54]:
# Realizar el cruce
merged_df_woden = merged_df_woden.merge(canal_alterno_cruce_inverso[['CLIENTE_ID', 'PROVEEDOR',"FECHA"]],on="CLIENTE_ID", how='left', suffixes=('', '_nuevo'))
# Actualizar la columna 'PROVEEDOR'
merged_df_woden['PROVEEDOR'] = merged_df_woden['PROVEEDOR_nuevo'].combine_first(merged_df_woden['PROVEEDOR'])
merged_df_woden=merged_df_woden.drop_duplicates(subset="CC-S")
# Eliminar la columna 'PROVEEDOR_nuevo' ya que no es necesaria después de la actualización
merged_df_woden = merged_df_woden.drop(columns=['PROVEEDOR_nuevo'])
merged_df_woden['PROVEEDOR'].value_counts(dropna=False)

PROVEEDOR
WODEN           96592
FIELD            4098
INTER            2587
Servientrega      716
Name: count, dtype: int64

In [55]:
merged_df_woden.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103993 entries, 0 to 118325
Data columns (total 30 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   TIPO BASE          103993 non-null  object        
 1   PEDIDO             103993 non-null  object        
 2   IDENTIFICADOR_ID   103993 non-null  object        
 3   CLIENTE_ID         103993 non-null  object        
 4   INTERFAZ           103993 non-null  object        
 5   CIUDAD             103993 non-null  object        
 6   DEPARTAMENTO       103993 non-null  object        
 7   MARCA_ID           103993 non-null  object        
 8   REFERENCIA_ID      103993 non-null  object        
 9   ID_EQUIPO          103993 non-null  object        
 10  SERIAL_EQUIPO      103993 non-null  object        
 11  MAC_EQUIPO         103993 non-null  object        
 12  PRODUCTO           103993 non-null  object        
 13  TECNOLOGÍA         103993 non-null  object       

In [56]:
# Filtrar las filas que no tienen las tipificaciones "RECUPERADO WODEN" y "ENTREGO A WODEN"
aplicar_logica = merged_df_woden[~merged_df_woden["PROVEEDOR"].isin(["WODEN"])]
aplicar_logica["PROVEEDOR"].value_counts(dropna=False)

PROVEEDOR
FIELD           4098
INTER           2587
Servientrega     716
Name: count, dtype: int64

In [57]:
aplicar_logica.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7401 entries, 85 to 118321
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          7401 non-null   object        
 1   PEDIDO             7401 non-null   object        
 2   IDENTIFICADOR_ID   7401 non-null   object        
 3   CLIENTE_ID         7401 non-null   object        
 4   INTERFAZ           7401 non-null   object        
 5   CIUDAD             7401 non-null   object        
 6   DEPARTAMENTO       7401 non-null   object        
 7   MARCA_ID           7401 non-null   object        
 8   REFERENCIA_ID      7401 non-null   object        
 9   ID_EQUIPO          7401 non-null   object        
 10  SERIAL_EQUIPO      7401 non-null   object        
 11  MAC_EQUIPO         7401 non-null   object        
 12  PRODUCTO           7401 non-null   object        
 13  TECNOLOGÍA         7401 non-null   object        
 14  UNIDAD_DE_

In [58]:
aplicar_logica['Diferencia_Dias'] = (merged_df_woden['FECHA_nuevo'] - merged_df_woden['FECHA_ENVIO IQE']).dt.days
aplicar_logica['Rango_Dias'] = aplicar_logica['Diferencia_Dias'].apply(lambda x: 'Si' if x >= -30 and x <= 60 else 'No')
aplicar_logica=aplicar_logica.drop_duplicates(subset="CC-S")
print(aplicar_logica.Rango_Dias.value_counts())

Rango_Dias
Si    6044
No    1357
Name: count, dtype: int64


In [59]:
sicumple=aplicar_logica[aplicar_logica['Rango_Dias']=="Si"]
sicumple["PROVEEDOR"].value_counts(dropna=False)

PROVEEDOR
FIELD           2918
INTER           2445
Servientrega     681
Name: count, dtype: int64

In [60]:
nocumple=aplicar_logica[aplicar_logica['Rango_Dias']=="No"]
nocumple["PROVEEDOR"].value_counts(dropna=False)

PROVEEDOR
FIELD           1180
INTER            142
Servientrega      35
Name: count, dtype: int64

In [ ]:
#primeraconcatenacion=pd.concat([sicumple,merged_df_no_woden,provedor_no_woden])
#primeraconcatenacion.info()

In [ ]:
#primeraconcatenacion["PROVEEDOR"].value_counts(dropna=False)

In [61]:
merged_df_woden["PROVEEDOR"].value_counts(dropna=False)

PROVEEDOR
WODEN           96592
FIELD            4098
INTER            2587
Servientrega      716
Name: count, dtype: int64

In [62]:
woden_vs_canal_alterno_serial=merged_df_woden[merged_df_woden["PROVEEDOR"]=="WODEN"]
woden_vs_canal_alterno_serial.info()

<class 'pandas.core.frame.DataFrame'>
Index: 96592 entries, 0 to 118325
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          96592 non-null  object        
 1   PEDIDO             96592 non-null  object        
 2   IDENTIFICADOR_ID   96592 non-null  object        
 3   CLIENTE_ID         96592 non-null  object        
 4   INTERFAZ           96592 non-null  object        
 5   CIUDAD             96592 non-null  object        
 6   DEPARTAMENTO       96592 non-null  object        
 7   MARCA_ID           96592 non-null  object        
 8   REFERENCIA_ID      96592 non-null  object        
 9   ID_EQUIPO          96592 non-null  object        
 10  SERIAL_EQUIPO      96592 non-null  object        
 11  MAC_EQUIPO         96592 non-null  object        
 12  PRODUCTO           96592 non-null  object        
 13  TECNOLOGÍA         96592 non-null  object        
 14  UNIDAD_DE_

In [63]:
# Realizar el cruce
woden_vs_canal_alterno_serial = woden_vs_canal_alterno_serial.merge(canal_alterno_cruce_inverso[['SERIALES', 'PROVEEDOR',"FECHA"]],left_on="SERIAL_EQUIPO",right_on="SERIALES", how='left', suffixes=('', '_nuevo'))
# Actualizar la columna 'PROVEEDOR'
woden_vs_canal_alterno_serial['PROVEEDOR'] = woden_vs_canal_alterno_serial['PROVEEDOR_nuevo'].combine_first(woden_vs_canal_alterno_serial['PROVEEDOR'])
woden_vs_canal_alterno_serial=woden_vs_canal_alterno_serial.drop_duplicates(subset="CC-S")
# Eliminar la columna 'PROVEEDOR_nuevo' ya que no es necesaria después de la actualización
woden_vs_canal_alterno_serial = woden_vs_canal_alterno_serial.drop(columns=['PROVEEDOR_nuevo'])
woden_vs_canal_alterno_serial['PROVEEDOR'].value_counts(dropna=False)

PROVEEDOR
WODEN           89299
INTER            3214
Servientrega     2176
FIELD            1903
Name: count, dtype: int64

In [64]:
woden_vs_canal_alterno_serial_distintoawoden=woden_vs_canal_alterno_serial[woden_vs_canal_alterno_serial['PROVEEDOR']!="WODEN"]
woden_vs_canal_alterno_serial_distintoawoden.drop(["LLAVE","FECHA_nuevo","SERIALES","FECHA_nuevo"],axis=1,inplace=True)
woden_vs_canal_alterno_serial_distintoawoden.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7293 entries, 0 to 97421
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          7293 non-null   object        
 1   PEDIDO             7293 non-null   object        
 2   IDENTIFICADOR_ID   7293 non-null   object        
 3   CLIENTE_ID         7293 non-null   object        
 4   INTERFAZ           7293 non-null   object        
 5   CIUDAD             7293 non-null   object        
 6   DEPARTAMENTO       7293 non-null   object        
 7   MARCA_ID           7293 non-null   object        
 8   REFERENCIA_ID      7293 non-null   object        
 9   ID_EQUIPO          7293 non-null   object        
 10  SERIAL_EQUIPO      7293 non-null   object        
 11  MAC_EQUIPO         7293 non-null   object        
 12  PRODUCTO           7293 non-null   object        
 13  TECNOLOGÍA         7293 non-null   object        
 14  UNIDAD_DE_NE

In [65]:
tipificaciones_canal_alterno=pd.concat([woden_vs_canal_alterno_serial_distintoawoden,sicumple,merged_df_no_woden,provedor_no_woden],ignore_index=True)
tipificaciones_canal_alterno.drop(["LLAVE","FECHA_nuevo","Diferencia_Dias","Rango_Dias"],axis=1,inplace=True)
tipificaciones_canal_alterno.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24052 entries, 0 to 24051
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          24052 non-null  object        
 1   PEDIDO             24052 non-null  object        
 2   IDENTIFICADOR_ID   24052 non-null  object        
 3   CLIENTE_ID         24052 non-null  object        
 4   INTERFAZ           24052 non-null  object        
 5   CIUDAD             24052 non-null  object        
 6   DEPARTAMENTO       24052 non-null  object        
 7   MARCA_ID           24052 non-null  object        
 8   REFERENCIA_ID      24052 non-null  object        
 9   ID_EQUIPO          24052 non-null  object        
 10  SERIAL_EQUIPO      24052 non-null  object        
 11  MAC_EQUIPO         24052 non-null  object        
 12  PRODUCTO           24052 non-null  object        
 13  TECNOLOGÍA         24052 non-null  object        
 14  UNIDAD

In [66]:
tipificaciones_canal_alterno["PROVEEDOR"].value_counts(dropna=False)

PROVEEDOR
Servientrega    10181
INTER            7628
FIELD            6243
Name: count, dtype: int64

In [68]:
cifras_recoleccion=pd.read_excel("C:/Users/oolea/Documents/01. Codigos PY/E2E- B2B/001. INFO OFICIAL ACTUALIZADA JUNIO/DATA E2E/cifras de recoleccion febrero-abril.xlsx")
print(cifras_recoleccion.info())
cifras_recoleccion.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157337 entries, 0 to 157336
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   SERIE      157337 non-null  object        
 1   GUIA       152393 non-null  object        
 2   FECHA      157336 non-null  datetime64[ns]
 3   MOTIVO II  157337 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 4.8+ MB
None


,SERIE,GUIA,FECHA,MOTIVO II
0,1CM4D96B7602758,4400139904,2024-02-24,FIELD
1,ACD4NJ697500845,4400139904,2024-02-24,FIELD
2,8852NB112100557,4400139904,2024-02-24,FIELD
3,B522220B9422,4400139904,2024-02-24,FIELD
4,B522192D3723,4400139904,2024-02-24,FIELD


In [70]:
cifras_recoleccion.columns=['SERIAL', 'GUIA', 'FECHA CIERRE', 'PROVEEDOR']

In [71]:
cifras_recoleccion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157337 entries, 0 to 157336
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   SERIAL        157337 non-null  object        
 1   GUIA          152393 non-null  object        
 2   FECHA CIERRE  157336 non-null  datetime64[ns]
 3   PROVEEDOR     157337 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 4.8+ MB


In [72]:
pro_woden=woden_vs_canal_alterno_serial[woden_vs_canal_alterno_serial["PROVEEDOR"]=="WODEN"]
pro_woden.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89299 entries, 1 to 97426
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          89299 non-null  object        
 1   PEDIDO             89299 non-null  object        
 2   IDENTIFICADOR_ID   89299 non-null  object        
 3   CLIENTE_ID         89299 non-null  object        
 4   INTERFAZ           89299 non-null  object        
 5   CIUDAD             89299 non-null  object        
 6   DEPARTAMENTO       89299 non-null  object        
 7   MARCA_ID           89299 non-null  object        
 8   REFERENCIA_ID      89299 non-null  object        
 9   ID_EQUIPO          89299 non-null  object        
 10  SERIAL_EQUIPO      89299 non-null  object        
 11  MAC_EQUIPO         89299 non-null  object        
 12  PRODUCTO           89299 non-null  object        
 13  TECNOLOGÍA         89299 non-null  object        
 14  UNIDAD_DE_N

In [73]:
# Realizar el cruce
pro_woden = pro_woden.merge(cifras_recoleccion[['SERIAL',"PROVEEDOR"]],left_on="SERIAL_EQUIPO",right_on="SERIAL", how='left', suffixes=('', '_nuevo'))
# Actualizar la columna 'PROVEEDOR'
pro_woden['PROVEEDOR'] = pro_woden['PROVEEDOR_nuevo'].combine_first(pro_woden['PROVEEDOR'])
pro_woden=pro_woden.drop_duplicates(subset="CC-S")
# Eliminar la columna 'PROVEEDOR_nuevo' ya que no es necesaria después de la actualización
pro_woden = pro_woden.drop(columns=['PROVEEDOR_nuevo'])
pro_woden['PROVEEDOR'].value_counts(dropna=False)

PROVEEDOR
WODEN              84012
FIELD               3888
SERVIENTREGA        1120
INTERRAPIDISIMO      279
Name: count, dtype: int64

In [74]:
pro_woden.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89299 entries, 0 to 89395
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          89299 non-null  object        
 1   PEDIDO             89299 non-null  object        
 2   IDENTIFICADOR_ID   89299 non-null  object        
 3   CLIENTE_ID         89299 non-null  object        
 4   INTERFAZ           89299 non-null  object        
 5   CIUDAD             89299 non-null  object        
 6   DEPARTAMENTO       89299 non-null  object        
 7   MARCA_ID           89299 non-null  object        
 8   REFERENCIA_ID      89299 non-null  object        
 9   ID_EQUIPO          89299 non-null  object        
 10  SERIAL_EQUIPO      89299 non-null  object        
 11  MAC_EQUIPO         89299 non-null  object        
 12  PRODUCTO           89299 non-null  object        
 13  TECNOLOGÍA         89299 non-null  object        
 14  UNIDAD_DE_N

In [75]:
pro_woden.drop(["LLAVE","FECHA_nuevo","SERIALES"],axis=1,inplace=True)
pro_woden.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89299 entries, 0 to 89395
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          89299 non-null  object        
 1   PEDIDO             89299 non-null  object        
 2   IDENTIFICADOR_ID   89299 non-null  object        
 3   CLIENTE_ID         89299 non-null  object        
 4   INTERFAZ           89299 non-null  object        
 5   CIUDAD             89299 non-null  object        
 6   DEPARTAMENTO       89299 non-null  object        
 7   MARCA_ID           89299 non-null  object        
 8   REFERENCIA_ID      89299 non-null  object        
 9   ID_EQUIPO          89299 non-null  object        
 10  SERIAL_EQUIPO      89299 non-null  object        
 11  MAC_EQUIPO         89299 non-null  object        
 12  PRODUCTO           89299 non-null  object        
 13  TECNOLOGÍA         89299 non-null  object        
 14  UNIDAD_DE_N

In [76]:
pro_woden.drop(["SERIAL"],axis=1,inplace=True)
pro_woden.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89299 entries, 0 to 89395
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          89299 non-null  object        
 1   PEDIDO             89299 non-null  object        
 2   IDENTIFICADOR_ID   89299 non-null  object        
 3   CLIENTE_ID         89299 non-null  object        
 4   INTERFAZ           89299 non-null  object        
 5   CIUDAD             89299 non-null  object        
 6   DEPARTAMENTO       89299 non-null  object        
 7   MARCA_ID           89299 non-null  object        
 8   REFERENCIA_ID      89299 non-null  object        
 9   ID_EQUIPO          89299 non-null  object        
 10  SERIAL_EQUIPO      89299 non-null  object        
 11  MAC_EQUIPO         89299 non-null  object        
 12  PRODUCTO           89299 non-null  object        
 13  TECNOLOGÍA         89299 non-null  object        
 14  UNIDAD_DE_N

In [77]:
nuevo=pro_woden[pro_woden["PROVEEDOR"]!="WODEN"]
nuevo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5287 entries, 20 to 89395
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          5287 non-null   object        
 1   PEDIDO             5287 non-null   object        
 2   IDENTIFICADOR_ID   5287 non-null   object        
 3   CLIENTE_ID         5287 non-null   object        
 4   INTERFAZ           5287 non-null   object        
 5   CIUDAD             5287 non-null   object        
 6   DEPARTAMENTO       5287 non-null   object        
 7   MARCA_ID           5287 non-null   object        
 8   REFERENCIA_ID      5287 non-null   object        
 9   ID_EQUIPO          5287 non-null   object        
 10  SERIAL_EQUIPO      5287 non-null   object        
 11  MAC_EQUIPO         5287 non-null   object        
 12  PRODUCTO           5287 non-null   object        
 13  TECNOLOGÍA         5287 non-null   object        
 14  UNIDAD_DE_N

In [78]:
tipificaciones_canal_alterno_nuevo=pd.concat([tipificaciones_canal_alterno,nuevo],ignore_index=True)
tipificaciones_canal_alterno_nuevo["PROVEEDOR"].value_counts(dropna=False)

PROVEEDOR
Servientrega       10181
FIELD              10131
INTER               7628
SERVIENTREGA        1120
INTERRAPIDISIMO      279
Name: count, dtype: int64

In [79]:
dicc={"Servientrega":"SERVIENTREGA",
      "INTER":"INTERRAPIDISIMO"
     }
tipificaciones_canal_alterno_nuevo['PROVEEDOR'] = tipificaciones_canal_alterno_nuevo['PROVEEDOR'].map(dicc).fillna(tipificaciones_canal_alterno_nuevo['PROVEEDOR'])
tipificaciones_canal_alterno_nuevo["PROVEEDOR"].value_counts(dropna=False)

PROVEEDOR
SERVIENTREGA       11301
FIELD              10131
INTERRAPIDISIMO     7907
Name: count, dtype: int64

In [80]:
nuevo_woden=pro_woden[pro_woden["PROVEEDOR"]=="WODEN"]
nuevo_woden.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84012 entries, 0 to 89394
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          84012 non-null  object        
 1   PEDIDO             84012 non-null  object        
 2   IDENTIFICADOR_ID   84012 non-null  object        
 3   CLIENTE_ID         84012 non-null  object        
 4   INTERFAZ           84012 non-null  object        
 5   CIUDAD             84012 non-null  object        
 6   DEPARTAMENTO       84012 non-null  object        
 7   MARCA_ID           84012 non-null  object        
 8   REFERENCIA_ID      84012 non-null  object        
 9   ID_EQUIPO          84012 non-null  object        
 10  SERIAL_EQUIPO      84012 non-null  object        
 11  MAC_EQUIPO         84012 non-null  object        
 12  PRODUCTO           84012 non-null  object        
 13  TECNOLOGÍA         84012 non-null  object        
 14  UNIDAD_DE_N

In [81]:
bases_enviadas_cruzo_nuevo=pd.concat([tipificaciones_canal_alterno_nuevo,nuevo_woden,filtrado2],ignore_index=True)
bases_enviadas_cruzo_nuevo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365149 entries, 0 to 365148
Data columns (total 28 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   TIPO BASE          365149 non-null  object        
 1   PEDIDO             365149 non-null  object        
 2   IDENTIFICADOR_ID   365149 non-null  object        
 3   CLIENTE_ID         365149 non-null  object        
 4   INTERFAZ           365149 non-null  object        
 5   CIUDAD             365149 non-null  object        
 6   DEPARTAMENTO       365149 non-null  object        
 7   MARCA_ID           365149 non-null  object        
 8   REFERENCIA_ID      365149 non-null  object        
 9   ID_EQUIPO          365149 non-null  object        
 10  SERIAL_EQUIPO      365149 non-null  object        
 11  MAC_EQUIPO         365149 non-null  object        
 12  PRODUCTO           365149 non-null  object        
 13  TECNOLOGÍA         365149 non-null  object  

In [82]:
bases_enviadas_cruzo_nuevo["PROVEEDOR"].value_counts(dropna=False)

PROVEEDOR
WODEN              335810
SERVIENTREGA        11301
FIELD               10131
INTERRAPIDISIMO      7907
Name: count, dtype: int64

In [76]:

df7=pd.read_parquet("C:/Users/oolea/Documents/01. Codigos PY/E2E- B2B/001. INFO OFICIAL ACTUALIZADA JUNIO/DATA E2E/DEPURACIONES - HISTORICO (E2E).parquet")
df7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3251088 entries, 0 to 3251087
Data columns (total 20 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   TIPO BASE          object
 1   PEDIDO             object
 2   IDENTIFICADOR_ID   object
 3   CLIENTE_ID         object
 4   INTERFAZ           object
 5   CIUDAD             object
 6   DEPARTAMENTO       object
 7   MARCA              object
 8   REFERENCIA         object
 9   ID_EQUIPO          object
 10  SERIAL_EQUIPO      object
 11  MAC_EQUIPO         object
 12  PRODUCTO           object
 13  TECNOLOGÍA         object
 14  UNIDAD_DE_NEGOCIO  object
 15  MACRO MOTIVO       object
 16  FECHA_ENVIO IQE    object
 17  REGIONAL           object
 18  MOTIVO             object
 19  NombreArchivo      object
dtypes: object(20)
memory usage: 496.1+ MB


In [77]:
df7['PC'] = df7['PEDIDO']+ df7['CLIENTE_ID']
df7['PCS'] = df7['PEDIDO']+ df7['CLIENTE_ID']+df7["SERIAL_EQUIPO"]
df7['CC-S'] = df7['CLIENTE_ID']+df7["SERIAL_EQUIPO"]


df7["PC"]=df7["PC"].str.upper()
df7["PCS"]=df7["PCS"].str.upper()
df7["CC-S"]=df7["CC-S"].str.upper()

depuraciones=df7.copy()

# Cruce no cruzo vs depuraciones

In [81]:
#creamos una columa llamada cruce con el valor fijo de NO CRUZO
concatenado_nocruzo["CRUCE"]="NO CRUZO"
concatenado_nocruzo.loc[concatenado_nocruzo["PCS"].isin(depuraciones["PCS"]), "CRUCE" ] = "CRUZO"
print(concatenado_nocruzo["CRUCE"].value_counts())

CRUCE
NO CRUZO    16067
CRUZO        1031
Name: count, dtype: int64


In [84]:
concatenado_nocruzo_pcs=concatenado_nocruzo.loc[concatenado_nocruzo["CRUCE"]=="NO CRUZO"]
concatenado_nocruzo_pcs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16067 entries, 0 to 17097
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          16067 non-null  object        
 1   PEDIDO             16067 non-null  object        
 2   IDENTIFICADOR_ID   16067 non-null  object        
 3   CLIENTE_ID         16067 non-null  object        
 4   INTERFAZ           16067 non-null  object        
 5   CIUDAD             16067 non-null  object        
 6   DEPARTAMENTO       16067 non-null  object        
 7   MARCA_ID           16067 non-null  object        
 8   REFERENCIA_ID      16067 non-null  object        
 9   ID_EQUIPO          16067 non-null  object        
 10  SERIAL_EQUIPO      16067 non-null  object        
 11  MAC_EQUIPO         16067 non-null  object        
 12  PRODUCTO           16067 non-null  object        
 13  TECNOLOGÍA         16067 non-null  object        
 14  UNIDAD_DE_N

In [86]:
CRUZO_pcs=concatenado_nocruzo[concatenado_nocruzo["CRUCE"]=="CRUZO"]
CRUZO_pcs.shape

(1031, 27)

In [87]:
CRUZO= pd.merge(CRUZO_pcs,
                     depuraciones[["PCS","MOTIVO"]],
                     left_on="PCS",right_on="PCS",how="inner")
CRUZO=CRUZO.drop_duplicates(subset="CC-S")
CRUZO.reset_index(drop=True,inplace=True)
print(CRUZO.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031 entries, 0 to 1030
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          1031 non-null   object        
 1   PEDIDO             1031 non-null   object        
 2   IDENTIFICADOR_ID   1031 non-null   object        
 3   CLIENTE_ID         1031 non-null   object        
 4   INTERFAZ           1031 non-null   object        
 5   CIUDAD             1031 non-null   object        
 6   DEPARTAMENTO       1031 non-null   object        
 7   MARCA_ID           1031 non-null   object        
 8   REFERENCIA_ID      1031 non-null   object        
 9   ID_EQUIPO          1031 non-null   object        
 10  SERIAL_EQUIPO      1031 non-null   object        
 11  MAC_EQUIPO         1031 non-null   object        
 12  PRODUCTO           1031 non-null   object        
 13  TECNOLOGÍA         1031 non-null   object        
 14  UNIDAD_D

In [88]:
CRUZO["MOTIVO"].value_counts()

MOTIVO
BASES ANTERIORES           608
TELEFONIA CON EQUIPO       152
INFORMACIÓN INCOMPLETA     150
FIELD                       51
WFSM                        44
OBSOLETOS                    9
TELEFONIA SIN EQUIPO         8
CANALES ALTERNOS             2
DESCARTES                    2
INTER-SERVI                  2
INTER                        1
SIN EQUIPO                   1
2. TELEFONIA CON EQUIPO      1
Name: count, dtype: int64

In [89]:
CRUZO["Tipo de cierre"]=CRUZO["MOTIVO"].copy()
CRUZO.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031 entries, 0 to 1030
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          1031 non-null   object        
 1   PEDIDO             1031 non-null   object        
 2   IDENTIFICADOR_ID   1031 non-null   object        
 3   CLIENTE_ID         1031 non-null   object        
 4   INTERFAZ           1031 non-null   object        
 5   CIUDAD             1031 non-null   object        
 6   DEPARTAMENTO       1031 non-null   object        
 7   MARCA_ID           1031 non-null   object        
 8   REFERENCIA_ID      1031 non-null   object        
 9   ID_EQUIPO          1031 non-null   object        
 10  SERIAL_EQUIPO      1031 non-null   object        
 11  MAC_EQUIPO         1031 non-null   object        
 12  PRODUCTO           1031 non-null   object        
 13  TECNOLOGÍA         1031 non-null   object        
 14  UNIDAD_D

In [90]:
CRUZO["PROVEEDOR"]="DESCARTADO"

In [92]:
CRUZO.drop(["MOTIVO"],axis=1,inplace= True)
CRUZO.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031 entries, 0 to 1030
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          1031 non-null   object        
 1   PEDIDO             1031 non-null   object        
 2   IDENTIFICADOR_ID   1031 non-null   object        
 3   CLIENTE_ID         1031 non-null   object        
 4   INTERFAZ           1031 non-null   object        
 5   CIUDAD             1031 non-null   object        
 6   DEPARTAMENTO       1031 non-null   object        
 7   MARCA_ID           1031 non-null   object        
 8   REFERENCIA_ID      1031 non-null   object        
 9   ID_EQUIPO          1031 non-null   object        
 10  SERIAL_EQUIPO      1031 non-null   object        
 11  MAC_EQUIPO         1031 non-null   object        
 12  PRODUCTO           1031 non-null   object        
 13  TECNOLOGÍA         1031 non-null   object        
 14  UNIDAD_D

In [93]:
#creamos una columa llamada cruce con el valor fijo de NO CRUZO
concatenado_nocruzo_pcs["CRUCE"]="NO CRUZO"
concatenado_nocruzo_pcs.loc[concatenado_nocruzo_pcs["CC-S"].isin(depuraciones["CC-S"]), "CRUCE" ] = "CRUZO"
print(concatenado_nocruzo_pcs["CRUCE"].value_counts())

CRUCE
NO CRUZO    15166
CRUZO         901
Name: count, dtype: int64


In [94]:
no_cruzo_ccs=concatenado_nocruzo_pcs.loc[concatenado_nocruzo_pcs["CRUCE"]=="NO CRUZO"]
no_cruzo_ccs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15166 entries, 0 to 17097
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          15166 non-null  object        
 1   PEDIDO             15166 non-null  object        
 2   IDENTIFICADOR_ID   15166 non-null  object        
 3   CLIENTE_ID         15166 non-null  object        
 4   INTERFAZ           15166 non-null  object        
 5   CIUDAD             15166 non-null  object        
 6   DEPARTAMENTO       15166 non-null  object        
 7   MARCA_ID           15166 non-null  object        
 8   REFERENCIA_ID      15166 non-null  object        
 9   ID_EQUIPO          15166 non-null  object        
 10  SERIAL_EQUIPO      15166 non-null  object        
 11  MAC_EQUIPO         15166 non-null  object        
 12  PRODUCTO           15166 non-null  object        
 13  TECNOLOGÍA         15166 non-null  object        
 14  UNIDAD_DE_N

In [95]:
CRUZO_ccs=concatenado_nocruzo_pcs[concatenado_nocruzo_pcs["CRUCE"]=="CRUZO"]
CRUZO_ccs.shape

(901, 27)

In [96]:
CRUZO2= pd.merge(CRUZO_ccs,
                     depuraciones[["CC-S","MOTIVO"]],
                     left_on="CC-S",right_on="CC-S",how="inner")
CRUZO2=CRUZO2.drop_duplicates(subset="CC-S")
CRUZO2["Tipo de cierre"]=CRUZO2["MOTIVO"].copy()
CRUZO2.reset_index(drop=True,inplace=True)
print(CRUZO2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 901 entries, 0 to 900
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          901 non-null    object        
 1   PEDIDO             901 non-null    object        
 2   IDENTIFICADOR_ID   901 non-null    object        
 3   CLIENTE_ID         901 non-null    object        
 4   INTERFAZ           901 non-null    object        
 5   CIUDAD             901 non-null    object        
 6   DEPARTAMENTO       901 non-null    object        
 7   MARCA_ID           901 non-null    object        
 8   REFERENCIA_ID      901 non-null    object        
 9   ID_EQUIPO          901 non-null    object        
 10  SERIAL_EQUIPO      901 non-null    object        
 11  MAC_EQUIPO         901 non-null    object        
 12  PRODUCTO           901 non-null    object        
 13  TECNOLOGÍA         901 non-null    object        
 14  UNIDAD_DE_

In [97]:
CRUZO2["PROVEEDOR"]="DESCARTADO"

In [98]:
CRUZO2.drop(["MOTIVO"],axis=1,inplace=True)
CRUZO2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 901 entries, 0 to 900
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          901 non-null    object        
 1   PEDIDO             901 non-null    object        
 2   IDENTIFICADOR_ID   901 non-null    object        
 3   CLIENTE_ID         901 non-null    object        
 4   INTERFAZ           901 non-null    object        
 5   CIUDAD             901 non-null    object        
 6   DEPARTAMENTO       901 non-null    object        
 7   MARCA_ID           901 non-null    object        
 8   REFERENCIA_ID      901 non-null    object        
 9   ID_EQUIPO          901 non-null    object        
 10  SERIAL_EQUIPO      901 non-null    object        
 11  MAC_EQUIPO         901 non-null    object        
 12  PRODUCTO           901 non-null    object        
 13  TECNOLOGÍA         901 non-null    object        
 14  UNIDAD_DE_

In [99]:
#creamos una columa llamada cruce con el valor fijo de NO CRUZO
no_cruzo_ccs["CRUCE"]="NO CRUZO"
no_cruzo_ccs.loc[no_cruzo_ccs["PC"].isin(depuraciones["PC"]), "CRUCE" ] = "CRUZO"
print(no_cruzo_ccs["CRUCE"].value_counts())

CRUCE
NO CRUZO    12100
CRUZO        3066
Name: count, dtype: int64


In [100]:
CRUZO_pc=no_cruzo_ccs[no_cruzo_ccs["CRUCE"]=="CRUZO"]
CRUZO_pc.shape

(3066, 27)

In [101]:
CRUZO3= pd.merge(CRUZO_pc,
                     depuraciones[["PC","MOTIVO"]],
                     left_on="PC",right_on="PC",how="inner")
CRUZO3=CRUZO3.drop_duplicates(subset="CC-S")
CRUZO3["Tipo de cierre"]=CRUZO3["MOTIVO"].copy()
CRUZO3["PROVEEDOR"]="DESCARTADO"
CRUZO3.reset_index(drop=True,inplace=True)
print(CRUZO3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066 entries, 0 to 3065
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          3066 non-null   object        
 1   PEDIDO             3066 non-null   object        
 2   IDENTIFICADOR_ID   3066 non-null   object        
 3   CLIENTE_ID         3066 non-null   object        
 4   INTERFAZ           3066 non-null   object        
 5   CIUDAD             3066 non-null   object        
 6   DEPARTAMENTO       3066 non-null   object        
 7   MARCA_ID           3066 non-null   object        
 8   REFERENCIA_ID      3066 non-null   object        
 9   ID_EQUIPO          3066 non-null   object        
 10  SERIAL_EQUIPO      3066 non-null   object        
 11  MAC_EQUIPO         3066 non-null   object        
 12  PRODUCTO           3066 non-null   object        
 13  TECNOLOGÍA         3066 non-null   object        
 14  UNIDAD_D

In [102]:
CRUZO3.drop(["MOTIVO"],axis=1,inplace=True)
CRUZO3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066 entries, 0 to 3065
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          3066 non-null   object        
 1   PEDIDO             3066 non-null   object        
 2   IDENTIFICADOR_ID   3066 non-null   object        
 3   CLIENTE_ID         3066 non-null   object        
 4   INTERFAZ           3066 non-null   object        
 5   CIUDAD             3066 non-null   object        
 6   DEPARTAMENTO       3066 non-null   object        
 7   MARCA_ID           3066 non-null   object        
 8   REFERENCIA_ID      3066 non-null   object        
 9   ID_EQUIPO          3066 non-null   object        
 10  SERIAL_EQUIPO      3066 non-null   object        
 11  MAC_EQUIPO         3066 non-null   object        
 12  PRODUCTO           3066 non-null   object        
 13  TECNOLOGÍA         3066 non-null   object        
 14  UNIDAD_D

In [105]:
no_cruzo_ccs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15166 entries, 0 to 17097
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          15166 non-null  object        
 1   PEDIDO             15166 non-null  object        
 2   IDENTIFICADOR_ID   15166 non-null  object        
 3   CLIENTE_ID         15166 non-null  object        
 4   INTERFAZ           15166 non-null  object        
 5   CIUDAD             15166 non-null  object        
 6   DEPARTAMENTO       15166 non-null  object        
 7   MARCA_ID           15166 non-null  object        
 8   REFERENCIA_ID      15166 non-null  object        
 9   ID_EQUIPO          15166 non-null  object        
 10  SERIAL_EQUIPO      15166 non-null  object        
 11  MAC_EQUIPO         15166 non-null  object        
 12  PRODUCTO           15166 non-null  object        
 13  TECNOLOGÍA         15166 non-null  object        
 14  UNIDAD_DE_N

In [106]:
no_cruzo_FINAL=no_cruzo_ccs[no_cruzo_ccs["CRUCE"]=="NO CRUZO"]
#no_cruzo_FINAL.drop(["PROVEEDOR"],axis=1,inplace=True)
no_cruzo_FINAL.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12100 entries, 8 to 17097
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          12100 non-null  object        
 1   PEDIDO             12100 non-null  object        
 2   IDENTIFICADOR_ID   12100 non-null  object        
 3   CLIENTE_ID         12100 non-null  object        
 4   INTERFAZ           12100 non-null  object        
 5   CIUDAD             12100 non-null  object        
 6   DEPARTAMENTO       12100 non-null  object        
 7   MARCA_ID           12100 non-null  object        
 8   REFERENCIA_ID      12100 non-null  object        
 9   ID_EQUIPO          12100 non-null  object        
 10  SERIAL_EQUIPO      12100 non-null  object        
 11  MAC_EQUIPO         12100 non-null  object        
 12  PRODUCTO           12100 non-null  object        
 13  TECNOLOGÍA         12100 non-null  object        
 14  UNIDAD_DE_N

In [107]:
no_cruzo_FINAL["CRUCE"]="NO CRUZO"
no_cruzo_FINAL["CRUCE"].value_counts()

CRUCE
NO CRUZO    12100
Name: count, dtype: int64

In [108]:
no_cruzo_FINAL.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12100 entries, 8 to 17097
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TIPO BASE          12100 non-null  object        
 1   PEDIDO             12100 non-null  object        
 2   IDENTIFICADOR_ID   12100 non-null  object        
 3   CLIENTE_ID         12100 non-null  object        
 4   INTERFAZ           12100 non-null  object        
 5   CIUDAD             12100 non-null  object        
 6   DEPARTAMENTO       12100 non-null  object        
 7   MARCA_ID           12100 non-null  object        
 8   REFERENCIA_ID      12100 non-null  object        
 9   ID_EQUIPO          12100 non-null  object        
 10  SERIAL_EQUIPO      12100 non-null  object        
 11  MAC_EQUIPO         12100 non-null  object        
 12  PRODUCTO           12100 non-null  object        
 13  TECNOLOGÍA         12100 non-null  object        
 14  UNIDAD_DE_N

In [110]:
CRUZO_FINAL=pd.concat([bases_enviadas_cruzo_nuevo,CRUZO,CRUZO2,CRUZO3],ignore_index=True)
#CRUZO_FINAL.drop(["Diferencia_Dias","Rango_Dias","FECHA_canal_alterno"],axis=1,inplace=True)
CRUZO_FINAL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370147 entries, 0 to 370146
Data columns (total 28 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   TIPO BASE          370147 non-null  object        
 1   PEDIDO             370147 non-null  object        
 2   IDENTIFICADOR_ID   370147 non-null  object        
 3   CLIENTE_ID         370147 non-null  object        
 4   INTERFAZ           370147 non-null  object        
 5   CIUDAD             370147 non-null  object        
 6   DEPARTAMENTO       370147 non-null  object        
 7   MARCA_ID           370147 non-null  object        
 8   REFERENCIA_ID      370147 non-null  object        
 9   ID_EQUIPO          370147 non-null  object        
 10  SERIAL_EQUIPO      370147 non-null  object        
 11  MAC_EQUIPO         370147 non-null  object        
 12  PRODUCTO           370147 non-null  object        
 13  TECNOLOGÍA         370147 non-null  object  

In [111]:
CRUZO_FINAL["CRUCE"]="CRUZO"
CRUZO_FINAL["CRUCE"].value_counts()

CRUCE
CRUZO    370147
Name: count, dtype: int64

In [112]:
concatenado_final=pd.concat([CRUZO_FINAL,no_cruzo_FINAL],ignore_index=True)
concatenado_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382247 entries, 0 to 382246
Data columns (total 28 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   TIPO BASE          382247 non-null  object        
 1   PEDIDO             382247 non-null  object        
 2   IDENTIFICADOR_ID   382247 non-null  object        
 3   CLIENTE_ID         382247 non-null  object        
 4   INTERFAZ           382247 non-null  object        
 5   CIUDAD             382247 non-null  object        
 6   DEPARTAMENTO       382247 non-null  object        
 7   MARCA_ID           382247 non-null  object        
 8   REFERENCIA_ID      382247 non-null  object        
 9   ID_EQUIPO          382247 non-null  object        
 10  SERIAL_EQUIPO      382247 non-null  object        
 11  MAC_EQUIPO         382247 non-null  object        
 12  PRODUCTO           382247 non-null  object        
 13  TECNOLOGÍA         382247 non-null  object  

In [113]:
concatenado_final['FECHA'].value_counts(dropna=False)

FECHA
nan                    369362
2024-06-10 00:00:00       810
2024-06-09 00:00:00       779
2024-02-12 00:00:00       627
2024-02-08 00:00:00       296
                        ...  
2023-05-18 00:00:00         1
2023-06-05 00:00:00         1
2023-09-27 00:00:00         1
2022-05-13 00:00:00         1
2024-03-13 07:50:10         1
Name: count, Length: 594, dtype: int64

In [115]:
concatenado_final['FECHA'] = pd.to_datetime(concatenado_final['FECHA'])

In [116]:
concatenado_final.to_parquet("concatenado_final_250624.parquet",index=False)